In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Adds parent folder (credit-risk-model) to sys.path


In [ ]:
import sys
import os

# Add project root to sys.path so `src` can be imported
sys.path.append(os.path.abspath(".."))

from src.feature_engineering import (
    add_aggregate_features,
    add_datetime_features,
    get_preprocessing_pipeline
)

import pandas as pd

In [4]:
from src.data_processing import build_feature_engineering_pipeline
df = pd.read_csv('../data/raw/data.csv')
pipeline = build_feature_engineering_pipeline()
df_transformed = pipeline.fit_transform(df)
print(df_transformed.head())


         TransactionId         BatchId       AccountId       SubscriptionId  \
0  TransactionId_76871   BatchId_36123  AccountId_3957   SubscriptionId_887   
1  TransactionId_73770   BatchId_15642  AccountId_4841  SubscriptionId_3829   
2  TransactionId_26203   BatchId_53941  AccountId_4229   SubscriptionId_222   
3    TransactionId_380  BatchId_102363   AccountId_648  SubscriptionId_2185   
4  TransactionId_28195   BatchId_38780  AccountId_4841  SubscriptionId_3829   

        CustomerId CurrencyCode  CountryCode    ProviderId     ProductId  \
0  CustomerId_4406          UGX          256  ProviderId_6  ProductId_10   
1  CustomerId_4406          UGX          256  ProviderId_4   ProductId_6   
2  CustomerId_4683          UGX          256  ProviderId_6   ProductId_1   
3   CustomerId_988          UGX          256  ProviderId_1  ProductId_21   
4   CustomerId_988          UGX          256  ProviderId_4   ProductId_6   

      ProductCategory  ... PricingStrategy  FraudResult  total_amoun

In [ ]:
df = add_aggregate_features(df)
df = add_datetime_features(df)

# (Optional) Handle missing values (drop or fill)
df = df.dropna()  # Or use df.fillna(method="ffill"), etc.

# Step 3: Build pipeline
pipeline = get_preprocessing_pipeline()

# Step 4: Fit the pipeline
X = pipeline.fit_transform(df)

# Step 5: Extract feature names
feature_names = get_feature_names(pipeline)

# Step 6: Convert to DataFrame
import numpy as np
X_df = pd.DataFrame(
    X.toarray() if hasattr(X, "toarray") else X,
    columns=feature_names
)

# Preview result
print("✅ Transformed shape:", X_df.shape)
X_df.head()

In [4]:
import pandas as pd
from src.feature_engineering import get_feature_engineering_pipeline


# Initialize pipeline
pipeline = get_feature_engineering_pipeline()

# Transform data
X = pipeline.fit_transform(df)

# (Optional) Inspect shape
print("Transformed shape:", X.shape)


Transformed shape: (95662, 29)


In [6]:
import pandas as pd
from src.feature_engineering import get_feature_engineering_pipeline

# Initialize pipeline
pipeline = get_feature_engineering_pipeline()

# Fit and transform data
X_transformed = pipeline.fit_transform(df)

# --- Get feature names ---
# 1. Extract numeric feature names (defined in your pipeline)
numeric_features = [
    "Amount", "PricingStrategy",
    "total_amount", "avg_amount", "count_transactions", "std_amount",
    "transaction_hour", "transaction_day", "transaction_month", "transaction_year"
]

# 2. Get OneHot encoded feature names
preprocessor = pipeline.named_steps['preprocessing']
onehot_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
categorical_features = ["ProductCategory", "ChannelId", "ProviderId"]
onehot_columns = onehot_encoder.get_feature_names_out(categorical_features)

# 3. Combine all features
all_features = numeric_features + list(onehot_columns)

# Print results
print("Transformed shape:", X_transformed.shape)
print("\nFeature names (first 10):", all_features[:10])
print("Total features:", len(all_features))

Transformed shape: (95662, 29)

Feature names (first 10): ['Amount', 'PricingStrategy', 'total_amount', 'avg_amount', 'count_transactions', 'std_amount', 'transaction_hour', 'transaction_day', 'transaction_month', 'transaction_year']
Total features: 29
